# Dataset

In this final project, you will analyze the performance of the 10 big fast-food companies in the stock market over the years. To do so, you will have to form groups of 5 people and give answers to all the questions provided below.

Each dataset contains 10 CSV files corresponding to a different fast-food company’s stock:

- **BRK-A**: Berkshire Hathaway Inc.
- **DNUT**: Krispy Kreme, Inc.
- **DPZ**: Domino’s Pizza, Inc.
- **LKNCY**: Luckin Coffee Inc.
- **MCD**: McDonald’s Corporation
- **PZZA**: Papa John’s International, Inc.
- **QSR**: Restaurant Brands International, Inc.
- **SBUX**: Starbucks Corporation
- **WEN**: The Wendy’s Corporation
- **YUM**: Yum! Brands, Inc.

## Columns

The datasets contain the following columns:

- **Date**: Trading date  
- **Open**: Opening price  
- **High**: Highest price during session  
- **Low**: Lowest price during session  
- **Close**: Closing price  
- **Adj Close**: Closing price adjusted for dividends and stock splits  
- **Volume**: Number of shares traded during the session


# Exercises

You will have to give answers to the following 10 exercises (1 point each):

**Preprocessing steps**

In [ ]:
# Impoprt all the libraries necesaires for the project
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

In [2]:
# Importing the 10 datasets for the exercises
# Create a dictionary to store all of them
all_datasets = {}
databases_path = './Databases' # The path where the datasets are located 

# Here we use for loop to iterate over the folder where the datasets are, extract them and store them in a dictionary. 
for database in os.listdir(databases_path):
    database_name = database.split('.')[0]
    df = pd.read_csv(os.path.join(databases_path, database))
    all_datasets[database_name] = df

1. **For each dataset**:
   - Print the number of rows and columns.
   - Display the column names and their data types.

2. **Extract all rows where the date is in the year 2023**:
   - Print the number of rows.
   - Visualize the Close price trend for this period.

3. **For each company**:
   - Find the day with the highest Close price and display it alongside the price.

4. **Group the data by month**:
   - Calculate the average Close price for each company.
   - Plot these monthly averages for 3 companies and compare them.
   - Justify the chart selection.

5. **For each company**:
   - Compute the yearly average of the Close price.
   - Plot a comparison of yearly averages for all companies on a chart.
   - Justify your chart selection.

6. **For each company**:
   - Create a plot showing the range of prices for each month.
   - Justify the chart selection.

7. **Create a plot showing the relationship between trading volume and the Close price**:
   - Select one company.
   - Add insights about the pattern.
   - Justify your chart selection.

8. **For each company**:
   - Identify the month with the highest total trading volume.
   - Display the results in a summary table, showing the month and total volume.

9. **Merge the datasets for all companies**:
   - Create a single dataset, one for each year.
   - Print the structure of the combined dataset, ensuring proper alignment and handling of missing values.

10. **For each company**:
    - Calculate the spread between the High and Low prices for each day.
    - Compute the average spread for each company.
    - Visualize the result in a chart and justify the chart selection.
    - Interpret which companies exhibit the largest spreads and why this might happen.